In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import norm
import datetime
import os
import supertrend_lib as stlib
import sbeta_lib as sbeta
import smacross_lib as smacross
import CAPM_lib as capm
import LR_run_model_lib as LR_run
import LR_model_lib as LR_models
import yfinance as yf
pd.set_option('display.max_rows', 200)

In [5]:
# symbols_list = eval_df.Stock.values
def get_company_info(symbols_list):
    info_list = list()
    
    for s in symbols_list:
        shandle = yf.Ticker(s)
        info_list.append(shandle.info)

    return info_list
    
# info_list = get_company_info(['aapl','msft'])
# print(info_list)

In [6]:
def add_update(eval_df,values):
    """ Add a a row to dataframe from a list of data points in the same sequence as the columns

    Args:
        eval_df (_type_): DataFrame receiving the new row
        values_list (_type_): A list of values in the same data types and sequance order as the dataframe columns

    Returns:
        _type_: The updated dataframe
    """
    tmp_df = pd.DataFrame([values], columns=eval_df.columns)
    eval_df = pd.concat([eval_df, tmp_df], ignore_index=True)
    return eval_df

In [7]:
def read_stocklist(filename):
    with open(filename) as f:
        stocklist = f.read()
    return stocklist.strip().split(",")


In [8]:
def update_models(stock_list):
    ret = LR_models.update_list_of_models(stock_list=stock_list)
    models_df = pd.DataFrame({'Stock' : stock_list,
                              'Model' : [f"Model{i}" for i in ret]})
    
    # print(models_df)
    return models_df

In [9]:
# stocks_from_file = read_stocklist(filename='./stocks_list.txt')
# stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
# dfff = update_models(stocks_from_file)
# dfff

In [10]:
# dfff[dfff['Stock'] == 'AAPL']['Model'].values[0]

In [11]:
def run_supertrend(stock,start_date):
    return stlib.supertrend(stock,start_date=start_date)


In [12]:
def init_eval_table():
    columns = ['Last_Run'          ,'Stock'             ,'Last_Price'     ,'%Std. Dev.','$Std. Dev.','Supertrend_Winner' ,'Supertrend_Result','ST_Signal_Date', 
               'LR_Best_Model'     ,'LR_Next_Day_Recomm', 'SMA_Crossed_Up','SMA_X_Date'        ,'SMA_FastXSlow'    , 'Beta'         , 
               'CAPM'              ,'Daily_VaR'  ]

    dtypes = ['datetime64[ns]'       , str                , float            ,str      ,str        , bool              , str               , str            , 
              str                  , str                , str              , 'datetime64[ns]'    , str               , float          , 
              float                , float        ]

    # Initialize the DataFrame with empty rows
    eval_df = pd.DataFrame(columns=columns)

    # Convert the column data types
    for col, dtype in zip(columns, dtypes):
        eval_df[col] = eval_df[col].astype(dtype)
        
    return eval_df

In [13]:
def first_date_N_years_ago(years):
    # Calculate start and end dates N years back
    end_date = datetime.date.today() - datetime.timedelta(days=1)
    start_date = end_date.replace(year=end_date.year - years, month=1, day=1)
    return start_date.strftime("%Y-%m-%d")


In [14]:
def load_sp500_list():
    stocks_list_csv = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp_df = pd.DataFrame()
    
    try:
        wiki_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Open the link and download S&P company details in a table
        data = wiki_data[0] # All data is stored in first cell
        sp_df = data.sort_values(by=['Symbol'], ascending=True) # Sort the dataframe on ticker in alphabetical ascending order
    except:
        print("Cannot open file", stocks_list_csv)

    # remove the dotted symbols, they are redundant 
    no_dot_symbols = [i for i in sp_df['Symbol'] if i.find('.')==-1]
    sp_df = sp_df[sp_df['Symbol'].isin(no_dot_symbols)]
    
    # returns count_row, count_col, df
    return sp_df.shape[0], sp_df.shape[1], sp_df


In [15]:
m,n,sp_df = load_sp500_list()
# sp_df[sp_df['Symbol'].isin([i for i in sp_df['Symbol'] if i.find('.')>0])]

In [16]:
def get_sigma(stock_data):
    # Calculate daily price changes
    price_changes = stock_data["Close"].pct_change()

    mu, std = norm.fit(price_changes.dropna())
    return mu, std

In [17]:
def recommendation_table(eval_df,stock_list, lookback_years=1, sma_fast=40, sma_slow=200,run_update_models=False):
    stock_list = [str(s).upper() for s in stock_list]
    # print("Stock List:",stock_list)
    start_date = first_date_N_years_ago(lookback_years)
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    if run_update_models :
        print("Updating Models ...",end='')
        stock_best_model_df = update_models(stock_list=stock_list)
        print("Done!")

    print('Performing Analysis and Recommendations ...',end='')

    for s in stock_list:
        winner,buysell,buysell_date,close_price,stock_data = run_supertrend(s,start_date)
        if(len(stock_data)<2):
            print(f"{s} data not found. Skipping!",end=",")
            continue
        
        print(f"{s}",end=',')
        mean, std_dev = get_sigma(stock_data)
        std_dev_pct = round(std_dev * 100,2)
        std_dev_dlr = round(close_price*std_dev,2)
        sma_sig, sma_date, fastXslow = smacross.sma_xing(stock_data,sma_fast,sma_slow)
        beta, market_data = sbeta.get_beta(stock_data)
        CAPM, VaR = capm.CAPM_VaR(stock_data=stock_data,market_data=market_data,bond_mat_duration = lookback_years,stock_beta=beta)
        LR_recommend = LR_run.get_recommendation(stock=s,lookback=lookback_years)
        LR_recommend_str = f"{LR_recommend[0]},{LR_recommend[1]},{LR_recommend[2]}"
        
        # columns = ['Last_Run'          ,'Stock'             ,'Last_Price'     ,'%Std. Dev.','$Std. Dev.','Supertrend_Winner' ,'Supertrend_Result','ST_Signal_Date', 
        #            'LR_Best_Model'     ,'LR_Next_Day_Recomm', 'SMA_Crossed_Up','SMA_X_Date'             ,'SMA_FastXSlow'     , 'Beta'         , 
        #            'CAPM'              ,'Daily_VaR'  ]
    
        if run_update_models:
            model_str = stock_best_model_df[stock_best_model_df['Stock'] == s].Model.values[0]
        else:
            model_str = 'N/A'
            
        new_row = [today           , s                  , close_price          ,f"+/-{std_dev_pct}%",f"+/-${std_dev_dlr}",    winner            , buysell         ,    buysell_date ,    
                   model_str       , LR_recommend_str   , sma_sig              , sma_date           , fastXslow          ,    beta              , 
                   CAPM            , VaR  ]    
        
        eval_df = add_update(eval_df=eval_df,values = new_row)
        
    print("Done!")
    return eval_df

In [18]:
def eval_all_sp500(lookback_years = 1,sma_fast = 40, sma_slow = 200, regenerate_models = False ):
    n,m,sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_sp500, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [19]:
def eval_list_from_file(filename='./stocks_list.txt',lookback_years = 1,sma_fast = 40, sma_slow = 200 ,regenerate_models = False):
    stocks_from_file = read_stocklist(filename=filename)
    stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
    today = datetime.datetime.today().strftime("%Y-%m-%d")

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_from_file, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [20]:
def screen_for_buys(eval_df, ignore_supertrend_winners=False):
        if not ignore_supertrend_winners:
                buys_df = eval_df[ (eval_df['Supertrend_Winner']==True) &  
                        (eval_df['Supertrend_Result']=='Buy') & 
                        (eval_df['LR_Next_Day_Recomm'] == 'Buy,Buy,Buy') &
                        (eval_df['SMA_Crossed_Up']=='Buy')].sort_values(by=['Supertrend_Winner','Supertrend_Result',
                                                                            'ST_Signal_Date','SMA_Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])
        else:
                buys_df = eval_df[ (eval_df['Supertrend_Result']=='Buy') & 
                        (eval_df['LR_Next_Day_Recomm'] == 'Buy,Buy,Buy') &
                        (eval_df['SMA_Crossed_Up']=='Buy')].sort_values(by=['Supertrend_Winner','Supertrend_Result',
                                                                            'ST_Signal_Date','SMA_Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])            
        
        return buys_df

In [21]:
results_dir = './eval_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
fname = f'{results_dir}/Eval_Results_{datetime.datetime.today().strftime("%Y_%m_%dT%I%M%S%p")}.csv'

### Evaluate Stocks List

In [24]:
regenerate_models = True
# symbols_file = './stocks_portfolio.txt'
symbols_file = './stocks_list.txt'
# symbols_file = './sectors_etfs.txt'

eval_df = eval_list_from_file(filename=symbols_file,lookback_years=2,regenerate_models=regenerate_models)
# eval_df_from_file = eval_df
# eval_df = eval_all_sp500(lookback_years=2, regenerate_models=regenerate_models)
eval_df.to_csv(fname)

Updating Models ...QQQ,AAPL,MSFT,NVDA,AMD,INTC,META,IBM,QCOM,MU,TSLA,SPY,STX,GOOGL,AMZN,SQ,COIN,MRVL,PLTR,AVGO,DE,CDNS,ADBE,NOW,SMCI,KLAC,TSM,ASML,LRCX,ON,TXN,AMAT,JBL,ADI,MCHP,SWKS,HON,DIA,Done!
Performing Analysis and Recommendations ...QQQ,AAPL,MSFT,NVDA,AMD,INTC,META,IBM,QCOM,MU,TSLA,SPY,STX,GOOGL,AMZN,SQ,COIN,MRVL,PLTR,AVGO,DE,CDNS,ADBE,NOW,SMCI,KLAC,TSM,ASML,LRCX,ON,TXN,AMAT,JBL,ADI,MCHP,SWKS,HON,DIA,Done!


### All Results

In [25]:
eval_df.sort_values(['ST_Signal_Date','SMA_X_Date'], ascending=False)

,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
30,2023-05-30,TXN,176.29,+/-1.8%,+/-$3.18,False,Buy,2023-05-26,Model2,"Sell,Sell,Sell",Buy,2022-12-13,"(40, 200)",1.1696,6.74,-4.28
33,2023-05-30,ADI,176.53,+/-2.0%,+/-$3.53,False,Sell,2023-05-24,Model2,"Buy,Buy,Buy",Buy,2022-12-14,"(40, 200)",1.2896,6.97,-4.76
34,2023-05-30,MCHP,78.32,+/-2.58%,+/-$2.02,False,Buy,2023-05-22,Model2,"Sell,Buy,Buy",Buy,2022-12-05,"(40, 200)",1.6668,7.67,-6.16
10,2023-05-30,TSLA,193.17,+/-3.86%,+/-$7.46,False,Buy,2023-05-22,Model2,"Buy,Buy,Buy",Sell,2022-10-03,"(40, 200)",1.8479,8.02,-9.26
22,2023-05-30,ADBE,415.39,+/-2.37%,+/-$9.86,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4624,7.29,-5.69
7,2023-05-30,IBM,128.89,+/-1.43%,+/-$1.84,False,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Sell,2023-03-09,"(40, 200)",0.5391,5.55,-3.42
21,2023-05-30,CDNS,228.33,+/-2.29%,+/-$5.22,True,Buy,2023-05-18,Model2,"Sell,Sell,Sell",Buy,2022-12-19,"(40, 200)",1.3880,7.15,-5.53
23,2023-05-30,NOW,537.46,+/-2.85%,+/-$15.34,False,Buy,2023-05-17,Model1,"Sell,Sell,Sell",Buy,2023-02-24,"(40, 200)",1.6420,7.63,-6.84
26,2023-05-30,TSM,103.21,+/-2.32%,+/-$2.39,False,Buy,2023-05-17,Model2,"Sell,Sell,Sell",Buy,2023-01-31,"(40, 200)",1.2162,6.83,-5.48
27,2023-05-30,ASML,735.93,+/-2.75%,+/-$20.24,True,Buy,2023-05-17,Model2,"Sell,Buy,Buy",Buy,2022-12-13,"(40, 200)",1.8077,7.94,-6.61


In [26]:
if symbols_file == './stocks_list.txt':
    eval_df.to_csv("./stocks_eval_df_table.csv")
elif symbols_file == './sectors_etfs.txt':
    eval_df.to_csv("./sectors_etfs_eval_df_table.csv")
    
print_it = False

def send_df_to_printer(df):
    pdf_filename = "full_eval_df_table.pdf"

    fig, ax =plt.subplots(figsize=(12,4))
    ax.axis('tight')
    ax.axis('off')
    the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='top')

    pp = PdfPages(pdf_filename)
    pp.savefig(fig, bbox_inches='tight')
    pp.close()

    os.system("lpr "+pdf_filename)
    
if (print_it):
    send_df_to_printer(eval_df)

### Buy, Buy, and more Buys

In [27]:
buys_eval_df= screen_for_buys(eval_df=eval_df,ignore_supertrend_winners=True)
print(f"{len(buys_eval_df)} Stocks:")
buys_eval_df

6 Stocks:


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
32,2023-05-30,JBL,91.57,+/-2.03%,+/-$1.86,True,Buy,2023-05-17,Model1,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2496,6.89,-4.95
3,2023-05-30,NVDA,389.46,+/-3.54%,+/-$13.78,True,Buy,2023-01-12,Model2,"Buy,Buy,Buy",Buy,2023-01-24,"(40, 200)",2.1785,8.64,-8.63
12,2023-05-30,STX,62.29,+/-2.56%,+/-$1.59,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.2186,6.83,-6.09
29,2023-05-30,ON,86.62,+/-3.32%,+/-$2.88,False,Buy,2023-05-01,Model1,"Buy,Buy,Buy",Buy,2022-08-19,"(40, 200)",1.9894,8.28,-8.11
14,2023-05-30,AMZN,120.11,+/-2.46%,+/-$2.95,False,Buy,2023-03-30,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.5088,7.38,-5.81
13,2023-05-30,GOOGL,124.61,+/-2.07%,+/-$2.58,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3474,7.07,-4.96


### Save the Buys

In [23]:
eval_df.to_csv("./stocks_buys_eval_df_table.csv")

### All Roads Lead to UP & Safe

In [28]:
buys_safe = buys_eval_df[(buys_eval_df['Beta']>1) & (buys_eval_df['Beta']<2) ].sort_values('Daily_VaR',ascending=False)
print(f"{len(buys_safe)} Stocks:")
buys_safe

5 Stocks:


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
32,2023-05-30,JBL,91.57,+/-2.03%,+/-$1.86,True,Buy,2023-05-17,Model1,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2496,6.89,-4.95
13,2023-05-30,GOOGL,124.61,+/-2.07%,+/-$2.58,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3474,7.07,-4.96
14,2023-05-30,AMZN,120.11,+/-2.46%,+/-$2.95,False,Buy,2023-03-30,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.5088,7.38,-5.81
12,2023-05-30,STX,62.29,+/-2.56%,+/-$1.59,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.2186,6.83,-6.09
29,2023-05-30,ON,86.62,+/-3.32%,+/-$2.88,False,Buy,2023-05-01,Model1,"Buy,Buy,Buy",Buy,2022-08-19,"(40, 200)",1.9894,8.28,-8.11


### Save the Safe-Buys as Top Picks

In [29]:
buys_safe.to_csv("./top_picks.csv")

### UP the Next Day

In [30]:
LR_Next_Day_Recomm_only = eval_df[eval_df['LR_Next_Day_Recomm']=='Buy,Buy,Buy'].sort_values('Daily_VaR',ascending=False)
print(f"{len(LR_Next_Day_Recomm_only)} Stocks:")
LR_Next_Day_Recomm_only

12 Stocks:


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
36,2023-05-30,HON,193.75,+/-1.37%,+/-$2.66,False,Buy,2023-04-28,Model1,"Buy,Buy,Buy",Sell,2023-03-31,"(40, 200)",0.8168,6.08,-3.24
33,2023-05-30,ADI,176.53,+/-2.0%,+/-$3.53,False,Sell,2023-05-24,Model2,"Buy,Buy,Buy",Buy,2022-12-14,"(40, 200)",1.2896,6.97,-4.76
20,2023-05-30,DE,357.21,+/-2.02%,+/-$7.23,True,Sell,2023-01-19,Model1,"Buy,Buy,Buy",Sell,2023-05-03,"(40, 200)",0.8260,6.09,-4.86
32,2023-05-30,JBL,91.57,+/-2.03%,+/-$1.86,True,Buy,2023-05-17,Model1,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2496,6.89,-4.95
13,2023-05-30,GOOGL,124.61,+/-2.07%,+/-$2.58,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3474,7.07,-4.96
5,2023-05-30,INTC,29.00,+/-2.25%,+/-$0.65,False,Sell,2023-04-24,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2392,6.87,-5.27
14,2023-05-30,AMZN,120.11,+/-2.46%,+/-$2.95,False,Buy,2023-03-30,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.5088,7.38,-5.81
8,2023-05-30,QCOM,110.35,+/-2.51%,+/-$2.77,False,Sell,2023-04-25,Model2,"Buy,Buy,Buy",Sell,2022-04-12,"(40, 200)",1.5276,7.41,-5.91
12,2023-05-30,STX,62.29,+/-2.56%,+/-$1.59,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.2186,6.83,-6.09
29,2023-05-30,ON,86.62,+/-3.32%,+/-$2.88,False,Buy,2023-05-01,Model1,"Buy,Buy,Buy",Buy,2022-08-19,"(40, 200)",1.9894,8.28,-8.11


### Down the Next Day

In [31]:
LR_Next_Day_Sell_only = eval_df[(eval_df['LR_Next_Day_Recomm']=='Sell,Buy,Buy') | 
                                (eval_df['LR_Next_Day_Recomm']=='Sell,Sell,Sell') | 
                                (eval_df['LR_Next_Day_Recomm']=='Sell,Buy,Sell') | 
                                (eval_df['LR_Next_Day_Recomm']=='Buy,Sell,Sell')]

print(f"{len(LR_Next_Day_Sell_only)} Stocks:")
LR_Next_Day_Sell_only

26 Stocks:


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
0,2023-05-30,QQQ,348.40,+/-1.6%,+/-$5.56,False,Buy,2023-03-21,Model2,"Buy,Sell,Sell",Buy,2023-03-01,"(40, 200)",1.2707,6.93,-3.79
1,2023-05-30,AAPL,175.43,+/-1.87%,+/-$3.28,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2948,6.98,-4.47
2,2023-05-30,MSFT,332.89,+/-1.84%,+/-$6.13,False,Buy,2023-03-16,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2533,6.90,-4.44
4,2023-05-30,AMD,127.03,+/-3.33%,+/-$4.22,True,Buy,2023-05-08,Model2,"Sell,Sell,Sell",Buy,2023-03-08,"(40, 200)",1.9642,8.23,-8.04
6,2023-05-30,META,262.04,+/-3.19%,+/-$8.37,True,Buy,2022-11-10,Model1,"Sell,Sell,Sell",Buy,2023-02-23,"(40, 200)",1.6847,7.71,-7.83
7,2023-05-30,IBM,128.89,+/-1.43%,+/-$1.84,False,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Sell,2023-03-09,"(40, 200)",0.5391,5.55,-3.42
9,2023-05-30,MU,73.93,+/-2.74%,+/-$2.03,False,Buy,2023-03-23,Model2,"Sell,Sell,Sell",Buy,2023-02-27,"(40, 200)",1.5175,7.39,-6.52
11,2023-05-30,SPY,420.02,+/-1.19%,+/-$4.99,False,Buy,2023-03-31,Model2,"Buy,Sell,Sell",Buy,2023-02-07,"(40, 200)",1.0026,6.43,-2.82
15,2023-05-30,SQ,59.00,+/-4.4%,+/-$2.6,False,Sell,2023-02-10,Model2,"Sell,Sell,Sell",Sell,2023-04-27,"(40, 200)",2.5445,9.33,-10.38
16,2023-05-30,COIN,56.92,+/-5.95%,+/-$3.38,False,Sell,2023-04-05,Model2,"Sell,Sell,Sell",Buy,2023-05-23,"(40, 200)",2.7455,-6.77,-14.29


### Supertrend Winners and Still Supertrending

In [32]:
up_Supertrend = eval_df[(eval_df['Supertrend_Result'] == 'Buy') & (eval_df['Supertrend_Winner'] == True)].sort_values('ST_Signal_Date', ascending=False)
print(f"{len(up_Supertrend)} Stocks:")
up_Supertrend

10 Stocks:


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
22,2023-05-30,ADBE,415.39,+/-2.37%,+/-$9.86,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4624,7.29,-5.69
21,2023-05-30,CDNS,228.33,+/-2.29%,+/-$5.22,True,Buy,2023-05-18,Model2,"Sell,Sell,Sell",Buy,2022-12-19,"(40, 200)",1.3880,7.15,-5.53
27,2023-05-30,ASML,735.93,+/-2.75%,+/-$20.24,True,Buy,2023-05-17,Model2,"Sell,Buy,Buy",Buy,2022-12-13,"(40, 200)",1.8077,7.94,-6.61
32,2023-05-30,JBL,91.57,+/-2.03%,+/-$1.86,True,Buy,2023-05-17,Model1,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2496,6.89,-4.95
4,2023-05-30,AMD,127.03,+/-3.33%,+/-$4.22,True,Buy,2023-05-08,Model2,"Sell,Sell,Sell",Buy,2023-03-08,"(40, 200)",1.9642,8.23,-8.04
24,2023-05-30,SMCI,219.18,+/-3.77%,+/-$8.26,True,Buy,2023-05-03,Model2,"Sell,Sell,Sell",None,None,"(40, 200)",1.3775,7.13,-9.09
1,2023-05-30,AAPL,175.43,+/-1.87%,+/-$3.28,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2948,6.98,-4.47
3,2023-05-30,NVDA,389.46,+/-3.54%,+/-$13.78,True,Buy,2023-01-12,Model2,"Buy,Buy,Buy",Buy,2023-01-24,"(40, 200)",2.1785,8.64,-8.63
6,2023-05-30,META,262.04,+/-3.19%,+/-$8.37,True,Buy,2022-11-10,Model1,"Sell,Sell,Sell",Buy,2023-02-23,"(40, 200)",1.6847,7.71,-7.83
19,2023-05-30,AVGO,812.73,+/-2.08%,+/-$16.91,True,Buy,2022-11-08,Model2,"Sell,Sell,Sell",Buy,2023-01-03,"(40, 200)",1.3444,7.07,-5.03


### Supertrending (Winners  or Not)

In [33]:
up_Supertrend = eval_df[eval_df['Supertrend_Result'] == 'Buy'].sort_values('ST_Signal_Date', ascending=False)
print(f"{len(up_Supertrend)} Stocks:")
up_Supertrend

30 Stocks:


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
30,2023-05-30,TXN,176.29,+/-1.8%,+/-$3.18,False,Buy,2023-05-26,Model2,"Sell,Sell,Sell",Buy,2022-12-13,"(40, 200)",1.1696,6.74,-4.28
34,2023-05-30,MCHP,78.32,+/-2.58%,+/-$2.02,False,Buy,2023-05-22,Model2,"Sell,Buy,Buy",Buy,2022-12-05,"(40, 200)",1.6668,7.67,-6.16
10,2023-05-30,TSLA,193.17,+/-3.86%,+/-$7.46,False,Buy,2023-05-22,Model2,"Buy,Buy,Buy",Sell,2022-10-03,"(40, 200)",1.8479,8.02,-9.26
22,2023-05-30,ADBE,415.39,+/-2.37%,+/-$9.86,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4624,7.29,-5.69
7,2023-05-30,IBM,128.89,+/-1.43%,+/-$1.84,False,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Sell,2023-03-09,"(40, 200)",0.5391,5.55,-3.42
21,2023-05-30,CDNS,228.33,+/-2.29%,+/-$5.22,True,Buy,2023-05-18,Model2,"Sell,Sell,Sell",Buy,2022-12-19,"(40, 200)",1.3880,7.15,-5.53
17,2023-05-30,MRVL,65.51,+/-3.64%,+/-$2.38,False,Buy,2023-05-17,Model2,"Sell,Sell,Sell",Sell,2022-04-08,"(40, 200)",2.1554,8.59,-8.64
26,2023-05-30,TSM,103.21,+/-2.32%,+/-$2.39,False,Buy,2023-05-17,Model2,"Sell,Sell,Sell",Buy,2023-01-31,"(40, 200)",1.2162,6.83,-5.48
23,2023-05-30,NOW,537.46,+/-2.85%,+/-$15.34,False,Buy,2023-05-17,Model1,"Sell,Sell,Sell",Buy,2023-02-24,"(40, 200)",1.6420,7.63,-6.84
27,2023-05-30,ASML,735.93,+/-2.75%,+/-$20.24,True,Buy,2023-05-17,Model2,"Sell,Buy,Buy",Buy,2022-12-13,"(40, 200)",1.8077,7.94,-6.61


### Fast SMA Crossed Slow SMA

In [34]:
Crossed_up = eval_df[eval_df['SMA_Crossed_Up'] == 'Buy'].sort_values('SMA_X_Date', ascending=False)
print(f"{len(Crossed_up)} Stocks")
Crossed_up

30 Stocks


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
14,2023-05-30,AMZN,120.11,+/-2.46%,+/-$2.95,False,Buy,2023-03-30,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.5088,7.38,-5.81
16,2023-05-30,COIN,56.92,+/-5.95%,+/-$3.38,False,Sell,2023-04-05,Model2,"Sell,Sell,Sell",Buy,2023-05-23,"(40, 200)",2.7455,-6.77,-14.29
12,2023-05-30,STX,62.29,+/-2.56%,+/-$1.59,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.2186,6.83,-6.09
5,2023-05-30,INTC,29.00,+/-2.25%,+/-$0.65,False,Sell,2023-04-24,Model2,"Buy,Buy,Buy",Buy,2023-05-01,"(40, 200)",1.2392,6.87,-5.27
13,2023-05-30,GOOGL,124.61,+/-2.07%,+/-$2.58,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3474,7.07,-4.96
18,2023-05-30,PLTR,13.65,+/-4.44%,+/-$0.61,False,Buy,2023-05-09,Model1,"Sell,Sell,Sell",Buy,2023-03-23,"(40, 200)",1.8129,7.95,-10.56
22,2023-05-30,ADBE,415.39,+/-2.37%,+/-$9.86,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4624,7.29,-5.69
2,2023-05-30,MSFT,332.89,+/-1.84%,+/-$6.13,False,Buy,2023-03-16,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2533,6.90,-4.44
1,2023-05-30,AAPL,175.43,+/-1.87%,+/-$3.28,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2948,6.98,-4.47
4,2023-05-30,AMD,127.03,+/-3.33%,+/-$4.22,True,Buy,2023-05-08,Model2,"Sell,Sell,Sell",Buy,2023-03-08,"(40, 200)",1.9642,8.23,-8.04


### SMA Crossed and in Supertrend

In [35]:
Crossed_up = eval_df[(eval_df['SMA_Crossed_Up'] == 'Buy') & (eval_df['Supertrend_Result'] == 'Buy') ].sort_values('SMA_X_Date', ascending=False)
print(f"{len(Crossed_up)} Stocks:")
Crossed_up

25 Stocks:


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
12,2023-05-30,STX,62.29,+/-2.56%,+/-$1.59,False,Buy,2023-05-15,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.2186,6.83,-6.09
14,2023-05-30,AMZN,120.11,+/-2.46%,+/-$2.95,False,Buy,2023-03-30,Model2,"Buy,Buy,Buy",Buy,2023-05-23,"(40, 200)",1.5088,7.38,-5.81
13,2023-05-30,GOOGL,124.61,+/-2.07%,+/-$2.58,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-04-25,"(40, 200)",1.3474,7.07,-4.96
18,2023-05-30,PLTR,13.65,+/-4.44%,+/-$0.61,False,Buy,2023-05-09,Model1,"Sell,Sell,Sell",Buy,2023-03-23,"(40, 200)",1.8129,7.95,-10.56
22,2023-05-30,ADBE,415.39,+/-2.37%,+/-$9.86,True,Buy,2023-05-19,Model2,"Sell,Sell,Sell",Buy,2023-03-16,"(40, 200)",1.4624,7.29,-5.69
2,2023-05-30,MSFT,332.89,+/-1.84%,+/-$6.13,False,Buy,2023-03-16,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2533,6.90,-4.44
1,2023-05-30,AAPL,175.43,+/-1.87%,+/-$3.28,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2948,6.98,-4.47
4,2023-05-30,AMD,127.03,+/-3.33%,+/-$4.22,True,Buy,2023-05-08,Model2,"Sell,Sell,Sell",Buy,2023-03-08,"(40, 200)",1.9642,8.23,-8.04
0,2023-05-30,QQQ,348.40,+/-1.6%,+/-$5.56,False,Buy,2023-03-21,Model2,"Buy,Sell,Sell",Buy,2023-03-01,"(40, 200)",1.2707,6.93,-3.79
9,2023-05-30,MU,73.93,+/-2.74%,+/-$2.03,False,Buy,2023-03-23,Model2,"Sell,Sell,Sell",Buy,2023-02-27,"(40, 200)",1.5175,7.39,-6.52


### Test Single Stock

In [36]:
stock_to_test = 'swks'
single_stock_df = init_eval_table()
single_stock_list = [stock_to_test]
single_stock_list = ['jbl','xxx','aapl']
single_stock_df = recommendation_table(single_stock_df,single_stock_list, lookback_years=5, sma_fast=40, sma_slow=200,run_update_models=True)
single_stock_df


Updating Models ...JBL,XXX,
1 Failed download:
- XXX: 1d data not available for startTime=-2208994789 and endTime=1685451200. Only 100 years worth of day granularity data are allowed to be fetched per request.
XXX data not found. Skipping,AAPL,Done!
Performing Analysis and Recommendations ...JBL,
1 Failed download:
- XXX: Period 'max' is invalid, must be one of ['1d', '5d']
XXX data not found. Skipping,XXX data not found. Skipping!,AAPL,Done!


,Last_Run,Stock,Last_Price,%Std. Dev.,$Std. Dev.,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
0,2023-05-30,JBL,91.57,+/-2.37%,+/-$2.17,True,Buy,2023-05-17,Model1,"Buy,Buy,Buy",Buy,2022-09-14,"(40, 200)",1.2744,12.94,-5.73
1,2023-05-30,AAPL,175.43,+/-2.07%,+/-$3.63,True,Buy,2023-01-23,Model2,"Sell,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2262,12.60,-5.03


In [33]:
import requests
import textwrap
nasdaq_api_key = 'yYCEL8BqzxYUsgG67FTb'

def get_company_info(ticker_symbol):
    ticker_symbol = str(ticker_symbol).upper()
    # Set the endpoint URL and the API key
    url = "https://api.nasdaq.com/api/company/{}/company-profile"
    api_key = nasdaq_api_key

    # Format the endpoint URL with the ticker symbol
    endpoint_url = url.format(ticker_symbol)

    # Set the headers and parameters for the request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299",
        "Accept-Language": "en-US,en;q=0.9"
    }

    params = {
        "apikey": nasdaq_api_key
    }

    # Send the request and get the response
    response = requests.get(endpoint_url, headers=headers, params=params)

    # Parse the response JSON
    response_json = response.json()

    # Get the company full name and sector
    dic = {
        "Ticker" : ticker_symbol,
        "Full_Name" : response_json["data"]["CompanyName"]['value'],
        "Sector" : response_json["data"]["Sector"]["value"],
        "Industry" : response_json["data"]["Industry"]["value"],
        "Region" : response_json["data"]["Region"]["value"],
        "Address" : response_json["data"]["Address"]["value"],
        "Description" : response_json["data"]["CompanyDescription"]["value"]
        }

    df = pd.DataFrame.from_dict(dic, orient='index').T
    return df

company_info = get_company_info('CHTR')
print(f"Company  : {company_info.loc[0]['Full_Name']}")
print(f"Sector   : {company_info.loc[0]['Sector']}")
print(f"Industry : {company_info.loc[0]['Industry']}")
print(f"Region   : {company_info.loc[0]['Region']}")

wrapped_text = textwrap.wrap(company_info.loc[0]['Description'],100)
for line in wrapped_text:
    print(line)


Company  : Charter Communications, Inc.
Sector   : Telecommunications
Industry : Cable & Other Pay Television Services
Region   : North America
Charter is the product of the 2016 merger of three cable companies, each with a decades-long history
in the business: Legacy Charter, Time Warner Cable, and Bright House Networks. The firm now holds
networks capable of providing television, internet access, and phone services to roughly 54 million
U.S. homes and businesses, around 40% of the country. Across this footprint, Charter serves 29
million residential and 2 million commercial customer accounts under the Spectrum brand, making it
the second-largest U.S. cable company behind Comcast. The firm also owns, in whole or in part,
sports and news networks, including Spectrum SportsNet (long-term local rights to Los Angeles Lakers
games), SportsNet LA (Los Angeles Dodgers), SportsNet New York (New York Mets), and Spectrum News
NY1.


In [34]:
import os
import json

def build_sp500_companies_database():
    n, m, sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]
    
    if not os.path.exists("./company_info"):
        os.makedirs("./company_info")
    
    for s in stocks_sp500:
        company_info = get_company_info(s)
        filename = f"./company_info/{s}.json"
        
        with open(filename, "w") as outfile:
            json.dump(company_info, outfile, indent=4)
        
        print(f"Saved {filename}")
    
    print("Done!")

# build_sp500_companies_database()

In [35]:
sp_df.tail(50)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
446,TXN,Texas Instruments,Information Technology,Semiconductors,"Dallas, Texas",NaN,97476,1930
447,TXT,Textron,Industrials,Aerospace & Defense,"Providence, Rhode Island",1978-12-31,217346,1923
456,TYL,Tyler Technologies,Information Technology,Application Software,"Plano, Texas",2020-06-22,860731,1966
462,UAL,United Airlines Holdings,Industrials,Passenger Airlines,"Chicago, Illinois",2015-09-03,100517,1967
459,UDR,"UDR, Inc.",Real Estate,Residential REITs,"Highlands Ranch, Colorado",2016-03-07,74208,1972
466,UHS,Universal Health Services,Health Care,Health Care Facilities,"King of Prussia, Pennsylvania",2014-09-20,352915,1979
460,ULTA,Ulta Beauty,Consumer Discretionary,Specialty Stores,"Bolingbrook, Illinois",2016-04-18,1403568,1990
465,UNH,UnitedHealth Group,Health Care,Managed Health Care,"Minnetonka, Minnesota",1994-07-01,731766,1977
461,UNP,Union Pacific Corporation,Industrials,Rail Transportation,"Omaha, Nebraska",1957-03-04,100885,1862
463,UPS,United Parcel Service,Industrials,Air Freight & Logistics,"Sandy Springs, Georgia",2002-07-22,1090727,1907
